In [ ]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `~/Documents/Study/Robotics/optimal_control_julia/lec_15/Project.toml`
Precompiling project...
  ✓ SnoopPrecompile
  ✓ Rmath_jll
  ✓ libblastrampoline_jll
  ✓ FFTW_jll
  ✓ ArrayInterfaceCore
  ✓ Rmath
  ✓ ArrayInterfaceGPUArrays
  ✓ ArrayInterfaceStaticArraysCore
  ✓ IterativeSolvers
  ✓ FiniteDifferences
  ✓ ArnoldiMethod
  ✓ ArrayInterface
  ✓ FiniteDiff
  ✓ CloseOpenIntervals
  ✓ ArrayInterfaceOffsetArrays
  ✓ BlockDiagonals
  ✓ ArrayInterfaceStaticArrays
  ✓ RecursiveArrayTools
  ✓ FFTW
  ✓ DualNumbers
  ✓ LayoutPointers
  ✓ HypergeometricFunctions
  ✓ Graphs
  ✓ StrideArraysCore
  ✓ VertexSafeGraphs
  ✓ ForwardDiff
  ✓ SciMLBase
  ✓ StatsFuns
  ✓ Polyester
  ✓ PreallocationTools
  ✓ NLSolversBase
  ✓ SparseDiffTools
  ✓ FastBroadcast
  ✓ SimpleNonlinearSolve
  ✓ Polynomials
  ✓ LineSearches
  ✓ RobotDynamics
  ✓ Distributions
  ✓ NLsolve
  ✓ MatrixPencils
  ✓ DSP
  ✓ TrajectoryOptimization
  ✓ RobotZoo
  ✓ ExponentialUtilities
  ✓ VectorizationBase
  ✓ 

In [ ]:
using LinearAlgebra
using ForwardDiff
using BlockDiagonals
using ControlSystems

In [ ]:
function hat(q)
    return [
        0 -q[3] q[2];
        q[3] 0 -q[1];
        -q[2] q[1] 0
    ]
end

function L(q)
    s = q[1]
    v = q[2:4]
    return [
        s -v';
        v s*I+hat(v)
    ]
end

In [ ]:
T = Diagonal([1, -1, -1, -1])
H = [ zeros(1,3)'; I(3) ]

In [ ]:
function G(q)
    return L(q)*H
end

function qtoQ(q)
    return H'*T*L(q)*T*L(q)*H
end
function rptoq(ϕ)
    return (1/sqrt(1+ϕ'*ϕ)) [
        1;
        ϕ
    ]
end
function qtorp(q)
    return q[2:4] / q[1]
end

In [ ]:
function E(q)
    return BlockDiagonal([ I(3), G(q), I(6) ])
end

## Quadrotor Parameters

In [ ]:
#Quadrotor parameters
m = 0.5
ℓ = 0.1750
J = Diagonal([0.0023, 0.0023, 0.004])
g = 9.81
kt=1.0
km=0.0245

h = 0.05 #20 Hz

Nx = 13     # number of states (quaternion)
Nx̃ = 12     # number of states (linearized)
Nu = 4     # number of controls
Tfinal = 5.0 # final time
Nt = Int(Tfinal/h)+1    # number of time steps
thist = Array(range(0,h*(Nt-1), step=h));